<a href="https://colab.research.google.com/github/theinem/FindMotif/blob/master/MotifsDetection_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Detección de Motifs V4**
Este notebook contiene los pasos para la proposición de motifs (motivos) candidatos dentro de un conjunto de cadenas de ADN.

Se analizará dormancy survival regulator (DosR). Este factor de transcripción regula la expresión de genes bajo condiciones hipóxicas para la turbeculosis micobacteriana. Se analizarán una serie de 15-mers para detectar los puntos de partida de los enlaces que disparan la actuación de DosR.

***La versión 3 emplea los seudo-contadores y una búsqueda aleatoria***

Mientras que el greedy search se queda en óptimos locales, la búsqueda aleatoria permite que puedan encontrarse con mayor probabilidad los óptimos globales. Siempre y cuando el número de iteraciones sea lo suficientemente alto.

La potencia de este método radica en que hay nuclétidos más frecuentes, así que un examen aleatorio tiene más posibilidades de encontrar iterativamente motifs cada vez mejores.

In [1]:
import random

### Montar el disco virtual

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

### Lectura del archivo

In [2]:
#Alternativa no explorada por el curso. Cargar los datos desde el disco.
#f = open('/content/drive/My Drive/Colab Notebooks/Bioinformatics/Semana 3/DosR.txt', 'r')
#Dna = f.read()

Dna = ["GCGCCCCGCCCGGACAGCCATGCGCTAACCCTGGCTTCGATGGCGCCGGCTCAGTTAGGGCCGGAAGTCCCCAATGTGGCAGACCTTTCGCCCCTGGCGGACGAATGACCCCAGTGGCCGGGACTTCAGGCCCTATCGGAGGGCTCCGGCGCGGTGGTCGGATTTGTCTGTGGAGGTTACACCCCAATCGCAAGGATGCATTATGACCAGCGAGCTGAGCCTGGTCGCCACTGGAAAGGGGAGCAACATC", "CCGATCGGCATCACTATCGGTCCTGCGGCCGCCCATAGCGCTATATCCGGCTGGTGAAATCAATTGACAACCTTCGACTTTGAGGTGGCCTACGGCGAGGACAAGCCAGGCAAGCCAGCTGCCTCAACGCGCGCCAGTACGGGTCCATCGACCCGCGGCCCACGGGTCAAACGACCCTAGTGTTCGCTACGACGTGGTCGTACCTTCGGCAGCAGATCAGCAATAGCACCCCGACTCGAGGAGGATCCCG", "ACCGTCGATGTGCCCGGTCGCGCCGCGTCCACCTCGGTCATCGACCCCACGATGAGGACGCCATCGGCCGCGACCAAGCCCCGTGAAACTCTGACGGCGTGCTGGCCGGGCTGCGGCACCTGATCACCTTAGGGCACTTGGGCCACCACAACGGGCCGCCGGTCTCGACAGTGGCCACCACCACACAGGTGACTTCCGGCGGGACGTAAGTCCCTAACGCGTCGTTCCGCACGCGGTTAGCTTTGCTGCC", "GGGTCAGGTATATTTATCGCACACTTGGGCACATGACACACAAGCGCCAGAATCCCGGACCGAACCGAGCACCGTGGGTGGGCAGCCTCCATACAGCGATGACCTGATCGATCATCGGCCAGGGCGCCGGGCTTCCAACCGTGGCCGTCTCAGTACCCAGCCTCATTGACCCTTCGACGCATCCACTGCGCGTAAGTCGGCTCAACCCTTTCAAACCGCTGGATTACCGACCGCAGAAAGGGGGCAGGAC", "GTAGGTCAAACCGGGTGTACATACCCGCTCAATCGCCCAGCACTTCGGGCAGATCACCGGGTTTCCCCGGTATCACCAATACTGCCACCAAACACAGCAGGCGGGAAGGGGCGAAAGTCCCTTATCCGACAATAAAACTTCGCTTGTTCGACGCCCGGTTCACCCGATATGCACGGCGCCCAGCCATTCGTGACCGACGTCCCCAGCCCCAAGGCCGAACGACCCTAGGAGCCACGAGCAATTCACAGCG", "CCGCTGGCGACGCTGTTCGCCGGCAGCGTGCGTGACGACTTCGAGCTGCCCGACTACACCTGGTGACCACCGCCGACGGGCACCTCTCCGCCAGGTAGGCACGGTTTGTCGCCGGCAATGTGACCTTTGGGCGCGGTCTTGAGGACCTTCGGCCCCACCCACGAGGCCGCCGCCGGCCGATCGTATGACGTGCAATGTACGCCATAGGGTGCGTGTTACGGCGATTACCTGAAGGCGGCGGTGGTCCGGA", "GGCCAACTGCACCGCGCTCTTGATGACATCGGTGGTCACCATGGTGTCCGGCATGATCAACCTCCGCTGTTCGATATCACCCCGATCTTTCTGAACGGCGGTTGGCAGACAACAGGGTCAATGGTCCCCAAGTGGATCACCGACGGGCGCGGACAAATGGCCCGCGCTTCGGGGACTTCTGTCCCTAGCCCTGGCCACGATGGGCTGGTCGGATCAAAGGCATCCGTTTCCATCGATTAGGAGGCATCAA", "GTACATGTCCAGAGCGAGCCTCAGCTTCTGCGCAGCGACGGAAACTGCCACACTCAAAGCCTACTGGGCGCACGTGTGGCAACGAGTCGATCCACACGAAATGCCGCCGTTGGGCCGCGGACTAGCCGAATTTTCCGGGTGGTGACACAGCCCACATTTGGCATGGGACTTTCGGCCCTGTCCGCGTCCGTGTCGGCCAGACAAGCTTTGGGCATTGGCCACAATCGGGCCACAATCGAAAGCCGAGCAG", "GGCAGCTGTCGGCAACTGTAAGCCATTTCTGGGACTTTGCTGTGAAAAGCTGGGCGATGGTTGTGGACCTGGACGAGCCACCCGTGCGATAGGTGAGATTCATTCTCGCCCTGACGGGTTGCGTCTGTCATCGGTCGATAAGGACTAACGGCCCTCAGGTGGGGACCAACGCCCCTGGGAGATAGCGGTCCCCGCCAGTAACGTACCGCTGAACCGACGGGATGTATCCGCCCCAGCGAAGGAGACGGCG", "TCAGCACCATGACCGCCTGGCCACCAATCGCCCGTAACAAGCGGGACGTCCGCGACGACGCGTGCGCTAGCGCCGTGGCGGTGACAACGACCAGATATGGTCCGAGCACGCGGGCGAACCTCGTGTTCTGGCCTCGGCCAGTTGTGTAGAGCTCATCGCTGTCATCGAGCGATATCCGACCACTGATCCAAGTCGGGGGCTCTGGGGACCGAAGTCCCCGGGCTCGGAGCTATCGGACCTCACGATCACC"]

### Definimos todas las funciones
Nos permitirán hacer una greedy search

### Ejecución de las funciones
Se buscará un 15-mer dentro de 10 segmentos de ADN.

In [26]:
def M_GibbsSampler(Dna, k, t, N, M):
  BestMotifs = [] # output variable
  BestMotifs = GibbsSampler(Dna, k, t, N)
  BestScore = Score(BestMotifs)
  for i in range(M-1):
      motifs = GibbsSampler(Dna, k, t, N)
      ActualScore = Score(motifs)
      if ActualScore < BestScore:
          BestMotifs = motifs
          BestScore = ActualScore
  return BestMotifs, BestScore


# Input:  Integers k, t, and N, followed by a collection of strings Dna
# Output: GibbsSampler(Dna, k, t, N)
def GibbsSampler(Dna, k, t, N):
    BestMotifs = [] # output variable
    motifs = RandomMotifs(Dna, k, t)
    BestMotifs = motifs.copy()
    BestScore = Score(BestMotifs)
    for j in range(N-1):
        i = random.randint(0,t-1)
        profile = Profile(motifs[:i] + motifs[i+1:])
        motifs[i] = ProfileGeneratedString(motifs[i], profile, k)
        ActualScore = Score(motifs)
        if ActualScore < BestScore:
            BestMotifs = motifs
            BestScore = ActualScore
    return BestMotifs

# place all subroutines needed for GibbsSampler below this line
def RandomMotifs(Dna, k, t):
    # place your code here.
    l = len(Dna[0])
    RandomMotif =[]
    for i in range(t):
        r = random.randint(0,l-k)
        RandomMotif.append(Dna[i][r:r+k])
    return RandomMotif

# Input:  A set of k-mers motifs
# Output: ProfileWithPseudocounts(Motifs)
def Profile(Motifs):
    profile = {} # output variable
    t = len(Motifs)+4
    k = len(Motifs[0])
    profile = Count(Motifs)
    for i in profile:  
        for j in range(k):
            profile[i][j] = profile[i][j]/t   
    return profile

# Input:  A set of k-mers motifs
# Output: CountWithPseudocounts(Motifs)
def Count(Motifs):
    count = {} # initializing the count dictionary
    k = len(Motifs[0])
    count = {'A':[1]*k,'C':[1]*k,'G':[1]*k,'T':[1]*k}
    
    t = len(Motifs)
    for i in range(t):
        for j in range(k):
            symbol = Motifs[i][j]
            count[symbol][j] += 1
    return count

def WeightedDie(Probabilities):
    # your code here
    n = random.uniform(0, 1)
    for p in Probabilities:
        n -= Probabilities[p]
        if n <= 0:
            return p

def Normalize(Probabilities):
    # your code here
    P_sum = sum(Probabilities.values())
    for k,v in Probabilities.items():
        Probabilities[k] = v/P_sum
    return Probabilities

def Pr(Text, Profile):
    p = float(1)
    for i in range(len(Text)):
        p *= Profile[Text[i]][i]
    return p

# Input:  A set of k-mers motifs
# Output: The score of these k-mers.
def Score(Motifs):
    result = Consensus(Motifs)
    counts = Count(Motifs)
    score = len(Motifs)*len(result)
    i = 0
    for symbol in result:
        # Score is the sum of (total number of elements per column MINUS
        # the number of occurence of the most frequent symbol per column)   
        score -= counts[symbol][i]
        i += 1
    return score

# Input:  A set of k-mers motifs
# Output: A consensus string of Motifs.
def Consensus(Motifs):
    k = len(Motifs[0])
    count = Count(Motifs)
    consensus = ""
    for j in range(k):
        m = 0
        frequentSymbol = ""
        for symbol in "ACGT":
            if count[symbol][j] > m:
                m = count[symbol][j]
                frequentSymbol = symbol
        consensus += frequentSymbol
    return consensus

# Input:  A string Text, a profile matrix Profile, and an integer k
# Output: ProfileGeneratedString(Text, profile, k)
def ProfileGeneratedString(Text, profile, k):
    # your code here
    n = len(Text)
    probabilities = {}
    for i in range(0,n-k+1):
        probabilities[Text[i:i+k]] = Pr(Text[i:i+k], profile)
    probabilities = Normalize(probabilities)
    return WeightedDie(probabilities)

In [33]:
# import the random package
import random
# Copy your GibbsSampler function (along with all required subroutines) below this line
def GibbsSampler(Dna, k, t, N):
    BestMotifs = [] # output variable
    # your code here
    Motifs = RandomMotifs(Dna, k, t)
    BestMotifs = Motifs
    for j in range(1,N):
        i = random.randint(0,t-1)
        ReducedMotifs = []
        for j in range(0,t):
            if j != i:
                ReducedMotifs.append(Motifs[j])
        Profile = ProfileWithPseudocounts(ReducedMotifs)
        Motif_i = ProfileGeneratedString(Dna[i], Profile, k)
        Motifs[i] = Motif_i
        if Score(Motifs) < Score(BestMotifs):
                BestMotifs=Motifs
    return BestMotifs

# place all subroutines needed for GibbsSampler below this line
# Input:  A list of strings Dna, and integers k and t
# Output: RandomMotifs(Dna, k, t)
# HINT:   You might not actually need to use t since t = len(Dna), but you may find it convenient

def RandomMotifs(Dna, k, t):
    # place your code here.
    s = len(Dna[0])
    rm = []
    for i in range(0,t):
        init_index = random.randint(1,s-k)
        rm.append(Dna[i][init_index:init_index+k])    
    return rm

# Input:  A set of kmers Motifs
# Output: ProfileWithPseudocounts(Motifs)

def ProfileWithPseudocounts(Motifs):
    t = len(Motifs)
    k = len(Motifs[0])
    profile = {} # output variable
    # your code here
    c = CountWithPseudocounts(Motifs)
    for n in 'ACGT':
        p = []
        for i in range(0,k):
            p.append(c[n][i]/(t+4))
        profile[n] = p
    return profile

# Input:  A set of kmers Motifs
# Output: CountWithPseudocounts(Motifs)

def CountWithPseudocounts(Motifs):
    t = len(Motifs)
    k = len(Motifs[0])
    # insert your code here
    count = {} # initializing the count dictionary
    for symbol in "ACGT":
        count[symbol] = []
        for j in range(k):
             count[symbol].append(1)
    for i in range(t):
        for j in range(k):
             symbol = Motifs[i][j]
             count[symbol][j] += 1
    return count 

#tests in which of the intervals defined by list ar the number r lies

def testinterval(ar,r):
    ar.sort()
    if r<= ar[0]:
      return ar[0]
    for i in range(1,len(ar)-1):
      if ar[i-1]<r<=ar[i]:
        return ar[i]
    if ar[len(ar)-2]< r:
      return ar[len(ar)-1]

# Input:  A dictionary Probabilities whose keys are k-mers and whose values are the probabilities of these kmers
# Output: A randomly chosen k-mer with respect to the values in Probabilities

def WeightedDie(Probabilities):
    # your code here
    sumprob = {}
    s = 0
    for p in Probabilities:
        s += Probabilities[p]
        sumprob[p] = s
    revprob = {}
    for q in sumprob:
      revprob[sumprob[q]] = q
    w = list(sumprob.values())
    r = random.uniform(0,1)
    kmer = revprob[testinterval(w,r)]
    return kmer

# Input:  A string Text, a profile matrix Profile, and an integer k
# Output: ProfileGeneratedString(Text, profile, k)

def ProfileGeneratedString(Text, profile, k):
    # your code here
    n = len(Text)
    probabilities = {} 
    for i in range(0,n-k+1):
        probabilities[Text[i:i+k]] = Pr(Text[i:i+k], profile)
    probabilities = Normalize(probabilities)
    return WeightedDie(probabilities)

# Input:  String Text and profile matrix Profile
# Output: Pr(Text, Profile)

def Pr(Text, Profile):
    # insert your code here
    p = 1
    for i in range(0,len(Text)):
        p *= Profile[Text[i]][i]
    return p

# Input: A dictionary Probabilities, where keys are k-mers and values are the probabilities of these k-mers (which do not necessarily sum up to 1)
# Output: A normalized dictionary where the probability of each k-mer was divided by the sum of all k-mers' probabilities

def Normalize(Probabilities):
    # your code here
    result = {}
    sum = 0
    for m in Probabilities:
        sum += Probabilities[m]
    for n in Probabilities:
        result[n]= Probabilities[n]/sum
    return result  

# Input:  A set of k-mers Motifs
# Output: The score of these k-mers.

def Score(Motifs):
    # Insert code here
    k = len(Motifs[0])
    t = len(Motifs)
    cs = ConsensusWithPseudocounts(Motifs)
    score = 0
    for j in range(0,k):
        for i in range(0,t):
            if Motifs[i][j] != cs[j]:
                score += 1
    return score

# Input:  A set of kmers Motifs
# Output: A consensus string of Motifs.

def ConsensusWithPseudocounts(Motifs):
    # insert your code here
    k = len(Motifs[0])
    count = CountWithPseudocounts(Motifs)
    consensus = ""
    for j in range(k):
        m = 0
        frequentSymbol = ""
        for symbol in "ACGT":
            if count[symbol][j] > m:
                m = count[symbol][j]
                frequentSymbol = symbol
        consensus += frequentSymbol
    return consensus    

In [38]:
t = len(Dna)
k = 15
N = 100
M = 40

BestMotifs, BestScore = M_GibbsSampler(Dna, k, t, N, M)

### Muestra de los motifs resultantes
En esta tercera versión los resultados pueden ser diversos, pero con 100 iteraciones tenderá a tener resultados similares a los obtenidos por la versión 2.

In [39]:
for i in range(len(BestMotifs)):
  print( i, '=', BestMotifs[i])

0 = GGGACTTCAGGCCCT
1 = GGGTCAAACGACCCT
2 = GGGACGTAAGTCCCT
3 = AGCGCCAGAATCCCG
4 = GGGGCGAAAGTCCCT
5 = AGGACCTTCGGCCCC
6 = GGGACTTCTGTCCCT
7 = GGGACTTTCGGCCCT
8 = GGGACCAACGCCCCT
9 = GGGACCGAAGTCCCC


### Consenso de los motifs y puntaje
El mejor puntaje posible es cero, es decir, se desea encontrar menor cantidad de diferencias entre los motifs propuestos.

Para N = 100 el score rondará entre 20 y 30.

In [40]:
print('Consenso:', Consensus(BestMotifs))
print('Puntaje:', Score(BestMotifs))

Consenso: GGGACCTAAGTCCCT
Puntaje: 36


# **---Con esto damos por terminado el código---**

# *Limitaciones*

*   Al no hacer una búsqueda exhaustiva, puede que no encuentre la cadena óptima global.

*   Los candidatos iniciales pueden funcionar bien en casos sencillos, pero este método puede fallar para problemas más complejos.

*   Algunos candidatos buenos pueden ser "olvidados" por el algoritmo, pues cada iteración es independiente.


Basado en los temas dictados en el curso Biology Meets Programming ( https://www.coursera.org/learn/bioinformatics/home/week/4 )

Otras fuentes: http://www.mrgraeme.co.uk/greedy-motif-search/